<a href="https://colab.research.google.com/github/jvataidee/SpatialDataSience/blob/master/Classificacao_ML_Usando_ArcGIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação com Machine Learning usando ArcGIS

In [ ]:
#Importar bibliotecas importantes
import arcpy
from arcpy import env
import os 
from arcpy.ia import *
env.overwriteOutput = True

In [ ]:
#Diretorio de workspace
workspace = r"F:\diretorio\model.gdb"
workspace2 = "F:\diretorio\model"

In [ ]:
#Diretorio das imagens
imagem_dir = r'F:\diretorio\model\cbers\CBERS_4A_WPM_20200528_203_142_L4'

banda0 = os.path.join(imagem_dir,r'CBERS_4A_WPM_20200528_203_142_L4_BAND0.tif')
banda1 = os.path.join(imagem_dir,r'CBERS_4A_WPM_20200528_203_142_L4_BAND1.tif')
banda2 = os.path.join(imagem_dir,r'CBERS_4A_WPM_20200528_203_142_L4_BAND2.tif')
banda3 = os.path.join(imagem_dir,r'CBERS_4A_WPM_20200528_203_142_L4_BAND3.tif')
banda4 = os.path.join(imagem_dir,r'CBERS_4A_WPM_20200528_203_142_L4_BAND4.tif')

In [ ]:
#Diretorio da composição
composicao_output = os.path.join(workspace2,"composicao.tif") 

In [ ]:
#Bandas para composição
composicao = r'{};{};{};{}'.format(banda4, banda2, banda1, banda3)
composicao

In [ ]:
#Realizar composição das bandas
arcpy.management.CompositeBands(composicao, composicao_output)

In [ ]:
#Diretorio do arquivo pansharpene
pan_output = os.path.join(workspace2,"pansharpene.tif")   

In [ ]:
#Realizar o Pansharpene
arcpy.management.CreatePansharpenedRasterDataset("composicao.tif",
                                                 1, 2, 3, 4,
                                                 pan_output,
                                                 banda0,
                                                 "Gram-Schmidt",
                                                 0.166, 0.167, 0.167, 0.5,
                                                 "UNKNOWN")

In [ ]:
#Diretorio do arquivo pansharpene
clip = os.path.join(workspace2,"clip.tif")   
ROI = os.path.join(workspace,"ROI") 

In [ ]:
# Realizar clip da imagem
arcpy.Clip_management(pan_output, "", 
                      clip, ROI, "0", "ClippingGeometry", 
                      "MAINTAIN_EXTENT")

## Classificando unsando ML

In [ ]:
#Diretório das amostras e do esquema
amostra_pixel = r'F:\diretorio\model.gdb\amostra'
output_ecd = os.path.join(workspace2,"TrainSVM.ecd")


In [ ]:
arcpy.ia.TrainSupportVectorMachineClassifier(clip,amostra_pixel,output_ecd,'',10,None, "Classvalue")

In [ ]:
#Diretório de classificação
classificado = os.path.join(workspace2,"classified.tif")

In [ ]:
out_raster_dataset = arcpy.ia.ClassifyRaster(clip, output_ecd, ''); out_raster_dataset.save(classificado)